In [1]:
# imports
from opm import OPM
from polygon import RESTClient
from credentials import POLYGON_API_KEY

In [19]:
# docs
# https://polygon.io/docs/options/get_v3_snapshot_options__underlyingasset___optioncontract
# https://polygon-api-client.readthedocs.io/en/latest/Snapshot.html

# client = RESTClient("XXXXXX") # hardcoded api_key is used
client = RESTClient(api_key=POLYGON_API_KEY)  # POLYGON_API_KEY environment variable is used
# Get options chain

underlying = "SPX"
today = "2023-08-25"

snapshot = client.get_snapshot_option("I:SPX", "O:SPXW230825P04500000")

# print raw values
print(snapshot)
def build_options_data(underlying, today):

    data = {'call': [], 'put': []}

    for c in client.list_snapshot_options_chain(underlying_asset=underlying, params={"expiration_date": today}):
        if c.greeks.delta == None or c.implied_volatility == None:
            continue
        data[c.details.contract_type].append({'ticker': c.details.ticker, 'iv': c.implied_volatility, 'delta': c.greeks.delta, 'expiration_date': c.details.expiration_date, 'strike_price': c.details.strike_price})
    
    return data

data = build_options_data(underlying="I:SPX", today=today)

OptionContractSnapshot(break_even_price=None, day=DayOptionContractSnapshot(change=-36, change_percent=-29.7, close=85.1, high=134.05, last_updated=1692921600000000000, low=84.3, open=100.2, previous_close=121.12, volume=79, vwap=106.8814), details=OptionDetails(contract_type='put', exercise_style='european', expiration_date='2023-08-25', shares_per_contract=100, strike_price=4500, ticker='O:SPXW230825P04500000'), greeks=Greeks(delta=-0.9863907485077884, gamma=0.0008890826946860184, theta=-2.7806084964603413, vega=0.053764845882433826), implied_volatility=0.37993328582016933, last_quote=LastQuoteOptionContractSnapshot(ask=86.3, ask_size=2, bid=85.8, bid_size=2, last_updated=1692990660011576832, midpoint=86.05, timeframe='REAL-TIME'), last_trade=LastTradeOptionContractSnapshot(price=86.03, sip_timestamp=1692990649283379456, size=14, conditions=[232], exchange=302, timeframe='REAL-TIME'), open_interest=5403, underlying_asset=UnderlyingAsset(change_to_break_even=None, last_updated=1680814

In [3]:
model = OPM(options_chain=data)

In [4]:
print(model.predict_delta(strike_price=4525))
print("actual delta: ", client.get_snapshot_option("I:SPX", "O:SPXW230825C04525000").greeks.delta)
print(model.predict_iv(strike_price=4525))
print("actual iv: ", client.get_snapshot_option("I:SPX", "O:SPXW230825C04525000").implied_volatility)

0.0021680209076545003
actual delta:  0.0021611932539323444
0.3578951485514405
actual iv:  0.3592234605507035


In [5]:
snap = client.get_snapshot_option("I:SPX", "O:SPXW230825C04525000")
snap

OptionContractSnapshot(break_even_price=None, day=DayOptionContractSnapshot(change=0, change_percent=0, close=0.05, high=0.05, last_updated=1692921600000000000, low=0.05, open=0.05, previous_close=0.05, volume=847, vwap=0.05), details=OptionDetails(contract_type='call', exercise_style='european', expiration_date='2023-08-25', shares_per_contract=100, strike_price=4525, ticker='O:SPXW230825C04525000'), greeks=Greeks(delta=0.0021540750617113575, gamma=0.00017553382725248745, theta=-0.6113172772792264, vega=0.008568009055365512), implied_volatility=0.3605497692866322, last_quote=LastQuoteOptionContractSnapshot(ask=0.05, ask_size=2313, bid=0, bid_size=0, last_updated=1692990077902421504, midpoint=0.025, timeframe='REAL-TIME'), last_trade=LastTradeOptionContractSnapshot(price=0.05, sip_timestamp=1692990007770146560, size=23, conditions=[232], exchange=302, timeframe='REAL-TIME'), open_interest=4568, underlying_asset=UnderlyingAsset(change_to_break_even=None, last_updated=1680814651655000000

In [6]:
model.strikes

array([1200, 1400, 1600, 1800, 2200, 2400, 2600, 4370, 4375, 4380, 4385,
       4390, 4395, 4400, 4405, 4410, 4415, 4420, 4425, 4430, 4435, 4440,
       4445, 4450, 4455, 4460, 4465, 4470, 4475, 4480, 4485, 4490, 4495,
       4500, 4505, 4510, 4515, 4520, 4525, 4530, 4535, 4540, 4545, 4550,
       4555, 4560, 4565, 4570, 4575, 4580, 4585, 4590, 4595, 4600, 4605,
       4610, 4615, 4620, 4625, 4630, 4635, 4640, 4645, 4650, 4655, 4660,
       4665, 4670, 4675, 4680, 4685, 4690, 4695, 4700, 4710, 4720, 4725,
       4730, 4740, 4750, 4760, 4770, 4775, 4780, 4790, 4800, 4810, 4820,
       4825, 4830, 4840, 4850, 4860, 4875, 4900, 4950, 5000, 5050, 5100,
       5200, 5400, 5600, 5800])

In [25]:
# computing probability of a close below 4405.69

"""
P(close below X) = 100 - Δ(X)
"""
# Get delta for 4405
delta = model.predict_delta(strike_price=4405)
print("strike-proximate P(close below X): ", 1 - delta)
delta = model.predict_delta(strike_price=4405)
print("model suggested P(close below X): ", 1 - delta)
print("model suggested bid: ", 1 - (1 - delta))

strike-proximate P(close below X):  0.24359470610869327
model suggested P(close below X):  0.24359470610869327
model suggested bid:  0.7564052938913067


In [23]:
model.get_delta_model_info()

{'MSE': 0.0, 'RMSE': 0.0}

In [20]:
# update models script
data = build_options_data(underlying="I:SPX", today=today)
model.update_models(options_chain=data)